In [3]:
from pulp import *

In [4]:
Warehouses = ["A", "B"]

supply = {"A": 1000, "B": 4000}

Bars = ["1", "2", "3", "4", "5"]

demand = {
    "1": 500,
    "2": 900,
    "3": 1800,
    "4": 200,
    "5": 700
}


In [5]:
costs = [  # Bars
    # 1 2 3 4 5
    [2, 4, 5, 2, 1],  # A   Warehouses
    [3, 1, 3, 2, 3],  # B
]
costs = makeDict([Warehouses, Bars], costs)
print(costs)

{'A': {'1': 2, '2': 4, '3': 5, '4': 2, '5': 1}, 'B': {'1': 3, '2': 1, '3': 3, '4': 2, '5': 3}}


In [35]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Beer Transporting Problem", LpMinimize)
Routes = []
for w in Warehouses:
    for b in Bars:
        Routes.append((w, b))
Routes

[('A', '1'),
 ('A', '2'),
 ('A', '3'),
 ('A', '4'),
 ('A', '5'),
 ('B', '1'),
 ('B', '2'),
 ('B', '3'),
 ('B', '4'),
 ('B', '5')]

In [36]:
vars = LpVariable.dicts("Route", (Warehouses, Bars), 0, None, LpInteger)
vars

{'A': {'1': Route_A_1,
  '2': Route_A_2,
  '3': Route_A_3,
  '4': Route_A_4,
  '5': Route_A_5},
 'B': {'1': Route_B_1,
  '2': Route_B_2,
  '3': Route_B_3,
  '4': Route_B_4,
  '5': Route_B_5}}

In [37]:
# the objective function
prob += (
    lpSum([costs[w][b] * vars[w][b] for (w, b) in Routes]),
    'Sum_of_Transporting_Costs'
)

# the Warehouses constraint
for w in Warehouses:
    prob += (
        lpSum([vars[w][b] for b in Bars]) <= supply[w], f"warehouse{w}capacity"
    )
for b in Bars:
    prob += (
        lpSum([vars[w][b] for w in Warehouses]) >= demand[b], f"Bar{b}capacity"
    )
# prob += (lpSum([vars[w][b] for (w, b) in Routes if w == 'A']) <= 1000, "WarehouseACapacity")
# prob += (lpSum([vars[w][b] for (w, b) in Routes if w == 'B']) <= 4000, "WarehouseBCapacity")

# prob += (lpSum([vars[w][b] for (w, b) in Routes if b == '1']) >= 500, "Bar1Capacity")
# prob += (lpSum([vars[w][b] for (w, b) in Routes if b == '2']) >= 900, "Bar2Capacity")
# prob += (lpSum([vars[w][b] for (w, b) in Routes if b == '3']) >= 1800, "Bar3Capacity")
# prob += (lpSum([vars[w][b] for (w, b) in Routes if b == '4']) >= 200, "Bar4Capacity")
# prob += (lpSum([vars[w][b] for (w, b) in Routes if b == '5']) >= 700, "Bar5Capacity")


In [38]:
prob.writeLP("BeerTransporting.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [39]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
# The optimised objective function value is printed to the screen
print("Total Cost of Transporting process = ", value(prob.objective))

Status: Optimal
Route_A_1 = 300.0
Route_A_2 = 0.0
Route_A_3 = 0.0
Route_A_4 = 0.0
Route_A_5 = 700.0
Route_B_1 = 200.0
Route_B_2 = 900.0
Route_B_3 = 1800.0
Route_B_4 = 200.0
Route_B_5 = 0.0
Total Cost of Transporting process =  8600.0
